In [1]:
import pandas as pd
import scanpy as sc
import anndata
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import qtl.norm
import itertools as it
import numba as nb

from os import makedirs, path
from typing import Union, List
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.sparse import csr_matrix

### Define functions

In [2]:
# ADPBulk from https://github.com/noamteyssier/adpbulk/blob/main/adpbulk/adpbulk.py

class ADPBulk:
    def __init__(
            self,
            adat: anndata.AnnData,
            groupby: Union[List[str], str],
            method: str = "sum",
            name_delim: str = "-",
            group_delim: str = ".",
            use_raw: bool = False):
        """
        Class of Pseudo-Bulking `AnnData` objects based on categorical variables
        found in the `.obs` attribute
        inputs:
            adat: anndata.AnnData
                The `AnnData` object to process
            groupby: Union[List[str], str]
                The categories to group by. Can provide as a single value
                or a list of values.
            method: str
                The method to aggregate with (sum[default], mean, median)
            name_delim: str
                The delimiter to use when grouping multiple categories together.
                example: 'cat1{delim}cat2'
            group_delim: str
                The delimiter to use for adding the value to its category.
                example: 'cat{delim}value'
            use_raw: bool
                Whether to use the `.raw` attribute on the `AnnData` object
        """

        self.agg_methods = {
            "sum": np.sum,
            "mean": np.mean,
            "median": np.median}

        self.adat = adat
        self.groupby = groupby
        self.method = method
        self.name_delim = name_delim
        self.group_delim = group_delim
        self.use_raw = use_raw

        self.group_idx = dict()
        self.groupings = list()
        self.grouping_masks = dict()

        self.meta = pd.DataFrame([])
        self.matrix = pd.DataFrame([])
        self.samples = pd.DataFrame([])

        self._isfit = False
        self._istransform = False

        self._validate()

    def _validate(self):
        """
        validates that the input is as expected
        """
        self._validate_anndata()
        self._validate_groups()
        self._validate_method()
        self._validate_raw()

    def _validate_anndata(self):
        """
        validates that the anndata object is as expected
        """
        if self.adat.X is None:
            raise AttributeError("Provided Matrix is None")
        if self.adat.obs is None:
            raise AttributeError("Provided Obs are None")
        if self.adat.var is None:
            raise AttributeError("Provided Var are None")

    def _validate_groups(self):
        """
        validates that the groups are as expected
        """
        # convert groups to list if provided as str
        if isinstance(self.groupby, str):
            self.groupby = [self.groupby]

        if isinstance(self.groupby, list):
            self.groupby = np.unique(self.groupby)
            for group in self.groupby:
                self._validate_group(group)
        else:
            raise TypeError("Groupby is not a list or str")

    def _validate_group(self, group):
        """
        confirms that provided group is a column in the observations
        """
        if group not in self.adat.obs.columns:
            raise ValueError(f"Provided group {group} not in observations")

    def _validate_method(self):
        """
        confirms that the method is known
        """
        if self.method not in self.agg_methods.keys():
            raise ValueError(
                f"Provided method {self.method} not in known methods {''.join(self.agg_methods)}")

    def _validate_raw(self):
        """
        if the `use_raw` flag is provided will confirm that
        the raw field is present
        """
        if self.use_raw and self.adat.raw is None:
            raise AttributeError(
                "use_raw provided, but no raw field is found in AnnData")


    def _fit_indices(self):
        """
        determines the indices for each of the provided groups
        """
        for group in self.groupby:
            unique_values = np.unique(self.adat.obs[group].values)
            self.group_idx[group] = {
                uv: set(np.flatnonzero(self.adat.obs[group].values == uv))
                    for uv in tqdm(unique_values, desc=f"fitting indices: {group}")}

    def _get_mask(self, pairs: tuple) -> np.ndarray:
        """
        retrieve the indices for the provided values from their respective groups
        calculates the global intersection between the sets
        """
        group_indices = []
        for j, key in enumerate(pairs):
            group_indices.append(self.group_idx[self.groupby[j]][key])
        mask = set.intersection(*group_indices)
        return np.array(list(mask))

    def _get_name(self, pairs: tuple) -> str:
        """
        create a name for the provided values based on their respective groups
        """
        name = self.name_delim.join([
                f"{self.groupby[i]}{self.group_delim}{pairs[i]}" for i in range(len(pairs))])
        return name

    def _get_agg(self, mask: np.ndarray) -> np.ndarray:
        """
        runs the aggregation function with the provided sample mask
        """
        if self.use_raw:
            mat = self.adat.raw.X[mask]
        else:
            mat = self.adat.X[mask]
        return self.agg_methods[self.method](mat, axis=0)

    def _get_var(self) -> np.ndarray:
        """
        return the var names using the given scheme (normal/raw)
        """
        if self.use_raw:
            return self.adat.raw.var.index.values
        else:
            return self.adat.var.index.values

    def _prepare_meta(self, pairs: tuple) -> dict:
        """
        defines the meta values for the pairs
        """
        values = {
            self.groupby[idx]: pairs[idx] for idx in np.arange(len(self.groupby))
            }
        values["SampleName"] = self._get_name(pairs)
        return values

    def _build_groupings(self):
        """
        generate the grouping iterable
        """
        if len(self.groupby) > 1:
            group_keys = [self.group_idx[g].keys() for g in self.groupby]
            self.groupings = list(it.product(*group_keys))
        else:
            self.groupings = self.group_idx[self.groupby[0]]

    def _build_masks(self):
        """
        generate the masks for each of the groupings and generates the
        metadata for each of the groupings
        """
        self.grouping_masks = dict()
        self.meta = []
        for pairs in self.groupings:
            if not isinstance(pairs, tuple):
                pairs = tuple([pairs])
            mask = self._get_mask(pairs)
            if mask.size > 0:
                self.grouping_masks[pairs] = mask
                self.meta.append(self._prepare_meta(pairs))

        if len(self.meta) == 0:
            raise ValueError("No combinations of the provided groupings found")
        self.meta = pd.DataFrame(self.meta)

    def fit(self):
        """
        fits the indices for each of the groups
        """
        self._fit_indices()
        self._build_groupings()
        self._build_masks()
        self._isfit = True

    def transform(self) -> pd.DataFrame:
        """
        performs the aggregation based on the fit indices
        """
        if not self._isfit:
            raise AttributeError("Please fit the object first")

        matrix = []
        for pairs in tqdm(self.groupings, desc="Aggregating Samples"):
            if not isinstance(pairs, tuple):
                pairs = tuple([pairs])
            if pairs in self.grouping_masks:
                matrix.append(self._get_agg(self.grouping_masks[pairs]))
        
        # stack all observations into single matrix
        matrix = np.vstack(matrix)

        self.matrix = pd.DataFrame(
            matrix,
            index=self.meta.SampleName.values,
            columns=self._get_var())

        self._istransform = True
        return self.matrix

    def fit_transform(self):
        """
        firs the indices and performs the aggregation based on those indices
        """
        self.fit()
        return self.transform()

    def get_meta(self) -> pd.DataFrame:
        """
        return the meta dataframe
        """
        if not self._isfit:
            raise AttributeError("Please fit the object first")
        return self.meta


def get_pca_linear_model_residuals(normalized_matrix):
    # scale data prior to PCA
    scaler = StandardScaler().set_output(transform="pandas")
    scaled_normalized_matrix = scaler.fit_transform(normalized_matrix)
    
    # fit PCA
    pca = PCA(n_components=10)
    pca_features = pca.fit_transform(scaled_normalized_matrix)
    
    # fit linear model
    reg = LinearRegression().fit(pca_features, normalized_matrix)
    predicted_vals = reg.predict(pca_features)

    # get residuals for log-TMM-normalized cell-type-specific pseudobulk
    model_residuals = normalized_matrix - predicted_vals

    # return transpose
    return model_residuals.T


def create_ct_df(in_df, cell_type, samples):
    ct_dic = {}
    df_cols = list(in_df)

    for s in samples:
        sample_col = f"cell_type_name.{cell_type}-donor_id.{s}"
        if sample_col in df_cols:
            ct_dic[s] = np.asarray(in_df[sample_col])
        else:
            ct_dic[s] = np.zeros(len(in_df))
            ct_dic[s][:] = np.nan

    ct_df = pd.DataFrame(ct_dic)
    ct_df.index = in_df.index
    return ct_df



def lupus_create_ct_df(in_df, cell_type, samples):
    ct_dic = {}
    df_cols = list(in_df)

    for s in samples:
        sample_col = f"ind_cov.{s}-onek1k_cell_type.{cell_type}"
        if sample_col in df_cols:
            ct_dic[s] = np.asarray(in_df[sample_col])
        else:
            ct_dic[s] = np.zeros(len(in_df))
            ct_dic[s][:] = np.nan

    ct_df = pd.DataFrame(ct_dic)
    ct_df.index = in_df.index
    return ct_df


# @nb.njit(parallel=True)
# def jit_inverse_normal_transform(M):
#     n_rows, n_cols = M.shape
#     Q = np.empty_like(M)
#     for i in nb.prange(n_rows):
#         R = np.argsort(np.argsort(M[i])) + 1
#         Q[i] = stats.norm.ppf(R / (n_cols + 1))
#     return Q

# def inverse_normal_transform_df(M_df):
#     M_array = M_df.to_numpy()  # Convert DataFrame to NumPy array
#     Q_array = jit_inverse_normal_transform(M_array)
#     return pd.DataFrame(Q_array, index=M_df.index, columns=M_df.columns)


def inverse_normal_transform(M):
    """Transform rows to a standard normal distribution"""
    R = stats.rankdata(M, axis=1)  # ties are averaged
    Q = stats.norm.ppf(R/(M.shape[1]+1))
    Q = pd.DataFrame(Q, index=M.index, columns=M.columns)
    return Q

### Load input data

In [3]:
# define input and output
base_dir = "/gpfs/commons/groups/gursoy_lab/cwalker/projects/sc_privacy/ml/"
data_dir = path.join(base_dir, "data")
cell_by_gene_h5_dir = path.join(data_dir, "onek1k", "cellbygene")
orig_cell_by_gene_h5_dir = path.join(data_dir, "onek1k", "adata")

cell_by_gene_h5_fi = path.join(cell_by_gene_h5_dir, "onek1k_cell_by_gene.h5ad")
orig_cell_by_gene_h5_fi = path.join(orig_cell_by_gene_h5_dir, "local.h5ad")

# output meta
out_metadata_dir = "../data/metadata/onek1k"
makedirs(out_metadata_dir, exist_ok=True)

In [ ]:
adata = anndata.read_h5ad(cell_by_gene_h5_fi)

In [7]:
orig_adata = anndata.read_h5ad(orig_cell_by_gene_h5_fi)

In [ ]:
for ct in list(set(orig_adata.obs["cell_type"])):
    orig_df = orig_adata.obs[orig_adata.obs["cell_type"] == ct].head(1)
    orig_cell_type = orig_df["cell_type"].item()
    orig_index = orig_df.index.item()
    new_name = adata.obs.loc[orig_index]["new_names"]

    print(f"{orig_cell_type}\t{new_name}")

In [7]:
# ct = "naive thymus-derived CD8-positive, alpha-beta T cell"
# orig_df = orig_adata.obs[orig_adata.obs["cell_type"] == ct]

row_keys = orig_adata.obs.index
filtered_row_keys = [key for key in row_keys if key in adata.obs.index]

# adata.loc[filtered_row_keys]
filtered_adata = adata[filtered_row_keys, :]
filtered_orig_adata = orig_adata[filtered_adata.obs.index, :]

assert filtered_adata.obs.index.equals(filtered_orig_adata.obs.index)

In [8]:
filtered_orig_adata.obs["cell_type_name"] = filtered_adata.obs["new_names"]

# filtered_orig_adata.obs.assign(cell_type_name=filtered_adata.obs.new_names)

/scratch/ipykernel_32271/2047994924.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  filtered_orig_adata.obs["cell_type_name"] = filtered_adata.obs["new_names"]


In [9]:
# adata = filtered_orig_adata

In [8]:
donors = list(set(adata.obs["donor_id"]))

In [9]:
retained_ensembl_ids = list(adata.var.index)
retained_gene_ids = list(adata.var["feature_name"])

In [10]:
ensembl_id_out = path.join(out_metadata_dir, "unfiltered_retained_ensembl_ids.txt")
gene_id_out = path.join(out_metadata_dir, "unfiltered_retained_gene_ids.txt")

with open(ensembl_id_out, "w") as f:
    f.write("\n".join(retained_ensembl_ids))

with open(gene_id_out, "w") as f:
    f.write("\n".join(retained_gene_ids))

In [11]:
adata.raw = adata

In [ ]:
reduced_cell_types = list(set(adata.obs["cell_type_name"]))

In [ ]:
adpb = ADPBulk(adata, ["donor_id", "cell_type_name"], use_raw=True, method="mean")
pseudobulk_matrix = adpb.fit_transform()

pseudobulk_matrix = pseudobulk_matrix.T
gene_names = adata.var.feature_name
pseudobulk_matrix.index = gene_names

Aggregating Samples: 100%|██████████| 15696/15696 [00:29<00:00, 534.00it/s] 


In [ ]:
for cell_type in reduced_cell_types:
    cell_type_cols = [col for col in pseudobulk_matrix if col.startswith(f"cell_type_name.{cell_type}-")]
    cell_type_pseudobulk_matrix = pseudobulk_matrix[cell_type_cols]
    print(cell_type, cell_type_pseudobulk_matrix.shape[1])

Platelets 761
Erythrocytes 210
CD4_NC 981
NK 981
B_MEM 981
NK_R 969
CD8_S100B 980
CD4_SOX4 855
CD4_ET 981
CD8_ET 981
Mono_NC 926
Mono_C 966
Plasma 791
B_IN 981
CD8_NC 981
DC 968


## Create sumagg data

In [16]:
# normalize pseudobulk gene counts by cell type and store in sample-specific dictionaries
# pseudobulk_matrix = pseudobulk_matrix.T

# dics for tmm-transformed data
gene_pseudobulk_dics = {}
gene_pseudobulk_invnorm_dics = {}
# for sample in donors:
#     gene_pseudobulk_dics[sample] = {}

gene_pseudobulk_residuals_dics = {}
# for sample in donors:
#     gene_pseudobulk_residuals_dics[sample] = {}
# # note the dic below is for tmm-transformed data with the RESIDUALS inv-norm
# # transformed
gene_pseudobulk_residuals_invnorm_transformed_dics = {}
# for ct in donors:
#     gene_pseudobulk_residuals_invnorm_transformed_dics[sample] = {}

for cell_type in tqdm(reduced_cell_types):
    # define cells
    cell_type_cols = [col for col in pseudobulk_matrix if col.startswith(f"cell_type_name.{cell_type}-")]
    # subset to keep only cells of cell type
    cell_type_pseudobulk_matrix = pseudobulk_matrix[cell_type_cols]
    # create log+1 pseudobulk matrix (sumagg)
    log_pseudobulk_matrix = np.log2(cell_type_pseudobulk_matrix + 1)
    # inverse normal transform the log pseudobulk matrix
    inv_norm_log_pseudobulk_matrix = qtl.norm.inverse_normal_transform(log_pseudobulk_matrix)
    # get residual expression using RINT-log pseudobulk matrix
    log_model_residuals = get_pca_linear_model_residuals(log_pseudobulk_matrix.T)
    # RINT the model residuals
    inv_norm_transformed_log_model_residuals = qtl.norm.inverse_normal_transform(log_model_residuals)
    # define matrices to store log pseudobulk, RINT-log-pseudobulk, and RINT-log-residual-pseudobulk
    df_columns = list(log_pseudobulk_matrix)
    log_pseudobulk_matrix_df = create_ct_df(
        log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_matrix_df = create_ct_df(
        inv_norm_log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_model_residuals_df = create_ct_df(
        inv_norm_transformed_log_model_residuals,
        cell_type,
        donors
    )
    # assign processed cell-type data to dictionaries
    gene_pseudobulk_dics[cell_type] = log_pseudobulk_matrix_df
    gene_pseudobulk_invnorm_dics[cell_type] = inv_norm_transformed_log_pseudobulk_matrix_df
    gene_pseudobulk_residuals_invnorm_transformed_dics[cell_type] = inv_norm_transformed_log_pseudobulk_model_residuals_df

100%|██████████| 16/16 [1:06:43<00:00, 250.20s/it]


In [19]:
out_cell_type_log_pseudobulk_dir = "../data/onek1k_cell_type_pseudobulk/sum_agg/log_pseudobulk_matrices"
makedirs(out_cell_type_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_dir = "../data/onek1k_cell_type_pseudobulk/sum_agg/invnorm_log_pseudobulk_matrices"
makedirs(out_cell_type_inv_norm_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_residuals_dir = "../data/onek1k_cell_type_pseudobulk/sum_agg/invnorm_regression_residuals"
makedirs(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, exist_ok=True)


for ct in reduced_cell_types:
    # process log tmm normalized
    gene_pseudobulk_dics[ct].to_csv(
        path.join(out_cell_type_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_invnorm_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_residuals_invnorm_transformed_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )


## Create log TMM data

In [ ]:
# normalize pseudobulk gene counts by cell type and store in sample-specific dictionaries
# pseudobulk_matrix = pseudobulk_matrix.T

# dics for tmm-transformed data
gene_pseudobulk_dics = {}
gene_pseudobulk_invnorm_dics = {}
# for sample in donors:
#     gene_pseudobulk_dics[sample] = {}

gene_pseudobulk_residuals_dics = {}
# for sample in donors:
#     gene_pseudobulk_residuals_dics[sample] = {}
# # note the dic below is for tmm-transformed data with the RESIDUALS inv-norm
# # transformed
gene_pseudobulk_residuals_invnorm_transformed_dics = {}
# for ct in donors:
#     gene_pseudobulk_residuals_invnorm_transformed_dics[sample] = {}

for cell_type in tqdm(reduced_cell_types):
    # define cells
    cell_type_cols = [col for col in pseudobulk_matrix if col.startswith(f"cell_type_name.{cell_type}-")]
    # subset to keep only cells of cell type
    cell_type_pseudobulk_matrix = pseudobulk_matrix[cell_type_cols]
    # create TMM matrix
    normalized_matrix = qtl.norm.edger_cpm(cell_type_pseudobulk_matrix, normalized_lib_sizes=True)
    # create log+1 pseudobulk matrix (sumagg)
    log_pseudobulk_matrix = np.log2(normalized_matrix + 1)
    # inverse normal transform the log pseudobulk matrix
    inv_norm_log_pseudobulk_matrix = qtl.norm.inverse_normal_transform(log_pseudobulk_matrix)
    # get residual expression using RINT-log pseudobulk matrix
    log_model_residuals = get_pca_linear_model_residuals(log_pseudobulk_matrix.T)
    # RINT the model residuals
    inv_norm_transformed_log_model_residuals = qtl.norm.inverse_normal_transform(log_model_residuals)
    # define matrices to store log pseudobulk, RINT-log-pseudobulk, and RINT-log-residual-pseudobulk
    df_columns = list(log_pseudobulk_matrix)
    log_pseudobulk_matrix_df = create_ct_df(
        log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_matrix_df = create_ct_df(
        inv_norm_log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_model_residuals_df = create_ct_df(
        inv_norm_transformed_log_model_residuals,
        cell_type,
        donors
    )
    # assign processed cell-type data to dictionaries
    gene_pseudobulk_dics[cell_type] = log_pseudobulk_matrix_df
    gene_pseudobulk_invnorm_dics[cell_type] = inv_norm_transformed_log_pseudobulk_matrix_df
    gene_pseudobulk_residuals_invnorm_transformed_dics[cell_type] = inv_norm_transformed_log_pseudobulk_model_residuals_df

In [ ]:
out_cell_type_log_pseudobulk_dir = "../data/onek1k_cell_type_pseudobulk/log_tmm/log_pseudobulk_matrices"
makedirs(out_cell_type_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_dir = "../data/onek1k_cell_type_pseudobulk/log_tmm/invnorm_log_pseudobulk_matrices"
makedirs(out_cell_type_inv_norm_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_residuals_dir = "../data/onek1k_cell_type_pseudobulk/log_tmm/invnorm_regression_residuals"
makedirs(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, exist_ok=True)


for ct in reduced_cell_types:
    # process log tmm normalized
    gene_pseudobulk_dics[ct].to_csv(
        path.join(out_cell_type_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_invnorm_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_residuals_invnorm_transformed_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )


# Lupus

### Load input data

In [3]:
lupus_integrated_h5 = "/gpfs/commons/groups/gursoy_lab/xli/Lupus/integration/data/Lupus_corrected_integrated.h5ad"

In [4]:
lupus_adata = sc.read_h5ad(lupus_integrated_h5)

In [5]:
lupus_adata.raw = lupus_adata

In [6]:
out_metadata_dir = "../data/onek1k_variant_selection/metadata/lupus"
makedirs(out_metadata_dir, exist_ok=True)

In [7]:
# extract lupus samples' auxiliary information and save to disk
columns_to_extract = {
    "age_sex_pop": ['ind_cov', 'Age', 'Sex', 'pop_cov'],
    "age_sex": ['ind_cov', 'Age', 'Sex'],
    "age_pop": ['ind_cov', 'Age', 'pop_cov'],
    "sex_pop": ['ind_cov', 'Sex', 'pop_cov'],
    "age": ['ind_cov', 'Age'],
    "sex": ['ind_cov', 'Sex'],
    "pop": ['ind_cov', 'pop_cov']
}
for col_set_name, cols in columns_to_extract.items():
    selected_data = lupus_adata.obs[cols]
    lupus_covariate_df = pd.DataFrame(selected_data)
    lupus_covariate_df.rename(
        columns={
            'ind_cov': 'sample', 'Age': 'age', 'Sex': 'sex', 'pop_cov': 'pop'
        },
    inplace=True
    )

    lupus_covariate_df = lupus_covariate_df.drop_duplicates(subset=['sample'])
    aux_out_path = path.join(out_metadata_dir, f"auxiliary_{col_set_name}.tsv")
    lupus_covariate_df.to_csv(aux_out_path, sep='\t', index=False)

In [8]:
# load CZI lupus adata
lupus_adata_file = "/gpfs/commons/groups/gursoy_lab/cwalker/projects/sc_privacy/data/lupus/adata_2/fd5e58b5-ddff-457f-b182-d18f99e36207.h5ad"
czi_lupus_adata = anndata.read_h5ad(lupus_adata_file)

In [11]:
donors = list(set(czi_lupus_adata.obs["ind_cov"]))

In [12]:
lupus_adata.obs.index = lupus_adata.obs.index.str.rstrip('-lupus')

In [13]:
assert all(lupus_adata.obs.index == czi_lupus_adata.obs.index)

In [14]:
czi_lupus_adata.obs["onek1k_cell_type"] = lupus_adata.obs["new_names"]

In [15]:
czi_lupus_adata.var

,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length
ENSG00000243485,True,MIR1302-2HG,NCBITaxon:9606,gene,1021
ENSG00000237613,True,FAM138A,NCBITaxon:9606,gene,1219
ENSG00000186092,True,OR4F5,NCBITaxon:9606,gene,2618
ENSG00000238009,True,RP11-34P13.7,NCBITaxon:9606,gene,3726
ENSG00000239945,True,RP11-34P13.8,NCBITaxon:9606,gene,1319
...,...,...,...,...,...
ENSG00000212907,True,MT-ND4L,NCBITaxon:9606,gene,297
ENSG00000198886,True,MT-ND4,NCBITaxon:9606,gene,1378
ENSG00000198786,True,MT-ND5,NCBITaxon:9606,gene,1812
ENSG00000198695,False,MT-ND6,NCBITaxon:9606,gene,525


In [16]:
retained_ensembl_ids = list(czi_lupus_adata.var.index)
retained_gene_ids = list(czi_lupus_adata.var["feature_name"])

ensembl_id_out = path.join(out_metadata_dir, "unfiltered_retained_ensembl_ids.txt")
gene_id_out = path.join(out_metadata_dir, "unfiltered_retained_gene_ids.txt")

with open(ensembl_id_out, "w") as f:
    f.write("\n".join(retained_ensembl_ids))

with open(gene_id_out, "w") as f:
    f.write("\n".join(retained_gene_ids))

In [17]:
czi_lupus_adata.raw.X.shape

(1263676, 30933)

In [18]:
(czi_lupus_adata.raw.X[0].toarray() == 0).sum()

30268

In [19]:
adpb = ADPBulk(czi_lupus_adata, ["ind_cov", "onek1k_cell_type"], use_raw=True, method="mean")
pseudobulk_matrix = adpb.fit_transform()

pseudobulk_matrix = pseudobulk_matrix.T
gene_names = czi_lupus_adata.var.feature_name
pseudobulk_matrix.index = gene_names

Aggregating Samples: 100%|██████████| 4176/4176 [00:32<00:00, 129.78it/s]


In [22]:
cell_types = list(set(czi_lupus_adata.obs["onek1k_cell_type"]))
print(cell_types)
reduced_cell_types = sorted(['Mono_NC', 'Plasma', 'CD8_ET', 'CD4_NC', 'Mono_C',
'DC', 'CD8_NC', 'NK_R', 'B_MEM', 'CD4_ET', 'CD8_S100B', 'B_IN', 'NK', 'CD4_SOX4'])

# reduced_cell_types = ['CD4_SOX4']

reduced_cell_types

['CD4_SOX4', 'CD4_NC', 'Plasma', 'Erythrocytes', 'CD8_ET', 'CD8_NC', 'Mono_C', 'CD8_S100B', 'DC', 'CD4_ET', 'NK_R', 'NK', 'Platelets', 'B_IN', 'B_MEM', 'Mono_NC']


['B_IN',
 'B_MEM',
 'CD4_ET',
 'CD4_NC',
 'CD4_SOX4',
 'CD8_ET',
 'CD8_NC',
 'CD8_S100B',
 'DC',
 'Mono_C',
 'Mono_NC',
 'NK',
 'NK_R',
 'Plasma']

In [23]:
for cell_type in reduced_cell_types:
    cell_type_cols = [col for col in pseudobulk_matrix if col.endswith(f"{cell_type}")]
    cell_type_pseudobulk_matrix = pseudobulk_matrix[cell_type_cols]
    print(cell_type, cell_type_pseudobulk_matrix.shape[1])

B_IN 261
B_MEM 261
CD4_ET 261
CD4_NC 261
CD4_SOX4 141
CD8_ET 261
CD8_NC 261
CD8_S100B 260
DC 259
Mono_C 261
Mono_NC 260
NK 261
NK_R 258
Plasma 215


## Create sumagg data

In [24]:
# normalize pseudobulk gene counts by cell type and store in sample-specific dictionaries
# pseudobulk_matrix = pseudobulk_matrix.T

# dics for tmm-transformed data
gene_pseudobulk_dics = {}
gene_pseudobulk_invnorm_dics = {}
# for sample in donors:
#     gene_pseudobulk_dics[sample] = {}

gene_pseudobulk_residuals_dics = {}
# for sample in donors:
#     gene_pseudobulk_residuals_dics[sample] = {}
# # note the dic below is for tmm-transformed data with the RESIDUALS inv-norm
# # transformed
gene_pseudobulk_residuals_invnorm_transformed_dics = {}
# for ct in donors:
#     gene_pseudobulk_residuals_invnorm_transformed_dics[sample] = {}

for cell_type in tqdm(reduced_cell_types):
    # define cells
    cell_type_cols = [col for col in pseudobulk_matrix if col.endswith(f"{cell_type}")]
    # subset to keep only cells of cell type
    cell_type_pseudobulk_matrix = pseudobulk_matrix[cell_type_cols]
    # create log+1 pseudobulk matrix (sumagg)
    log_pseudobulk_matrix = np.log2(cell_type_pseudobulk_matrix + 1)
    # inverse normal transform the log pseudobulk matrix
    inv_norm_log_pseudobulk_matrix = qtl.norm.inverse_normal_transform(log_pseudobulk_matrix)
    # get residual expression using RINT-log pseudobulk matrix
    log_model_residuals = get_pca_linear_model_residuals(log_pseudobulk_matrix.T)
    # RINT the model residuals
    inv_norm_transformed_log_model_residuals = qtl.norm.inverse_normal_transform(log_model_residuals)
    # define matrices to store log pseudobulk, RINT-log-pseudobulk, and RINT-log-residual-pseudobulk
    df_columns = list(log_pseudobulk_matrix)
    log_pseudobulk_matrix_df = lupus_create_ct_df(
        log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_matrix_df = lupus_create_ct_df(
        inv_norm_log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_model_residuals_df = lupus_create_ct_df(
        inv_norm_transformed_log_model_residuals,
        cell_type,
        donors
    )
    # assign processed cell-type data to dictionaries
    gene_pseudobulk_dics[cell_type] = log_pseudobulk_matrix_df
    gene_pseudobulk_invnorm_dics[cell_type] = inv_norm_transformed_log_pseudobulk_matrix_df
    gene_pseudobulk_residuals_invnorm_transformed_dics[cell_type] = inv_norm_transformed_log_pseudobulk_model_residuals_df


out_cell_type_log_pseudobulk_dir = "../data/lupus_cell_type_pseudobulk/sum_agg/log_pseudobulk_matrices"
makedirs(out_cell_type_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_dir = "../data/lupus_cell_type_pseudobulk/sum_agg/invnorm_log_pseudobulk_matrices"
makedirs(out_cell_type_inv_norm_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_residuals_dir = "../data/lupus_cell_type_pseudobulk/sum_agg/invnorm_regression_residuals"
makedirs(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, exist_ok=True)


for ct in tqdm(reduced_cell_types):
    # process log tmm normalized
    gene_pseudobulk_dics[ct].to_csv(
        path.join(out_cell_type_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_invnorm_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_residuals_invnorm_transformed_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )


100%|██████████| 14/14 [14:27<00:00, 61.98s/it]


## Create log TMM data

In [25]:
# normalize pseudobulk gene counts by cell type and store in sample-specific dictionaries
# pseudobulk_matrix = pseudobulk_matrix.T

# dics for tmm-transformed data
gene_pseudobulk_dics = {}
gene_pseudobulk_invnorm_dics = {}
# for sample in donors:
#     gene_pseudobulk_dics[sample] = {}

gene_pseudobulk_residuals_dics = {}
# for sample in donors:
#     gene_pseudobulk_residuals_dics[sample] = {}
# # note the dic below is for tmm-transformed data with the RESIDUALS inv-norm
# # transformed
gene_pseudobulk_residuals_invnorm_transformed_dics = {}
# for ct in donors:
#     gene_pseudobulk_residuals_invnorm_transformed_dics[sample] = {}

for cell_type in tqdm(reduced_cell_types):
    # define cells
    cell_type_cols = [col for col in pseudobulk_matrix if col.endswith(f"{cell_type}")]
    # subset to keep only cells of cell type
    cell_type_pseudobulk_matrix = pseudobulk_matrix[cell_type_cols]
    # create TMM matrix
    normalized_matrix = qtl.norm.edger_cpm(cell_type_pseudobulk_matrix, normalized_lib_sizes=True)
    # create log+1 pseudobulk matrix (sumagg)
    log_pseudobulk_matrix = np.log2(normalized_matrix + 1)
    # inverse normal transform the log pseudobulk matrix
    inv_norm_log_pseudobulk_matrix = qtl.norm.inverse_normal_transform(log_pseudobulk_matrix)
    # get residual expression using RINT-log pseudobulk matrix
    log_model_residuals = get_pca_linear_model_residuals(log_pseudobulk_matrix.T)
    # RINT the model residuals
    inv_norm_transformed_log_model_residuals = qtl.norm.inverse_normal_transform(log_model_residuals)
    # define matrices to store log pseudobulk, RINT-log-pseudobulk, and RINT-log-residual-pseudobulk
    df_columns = list(log_pseudobulk_matrix)
    log_pseudobulk_matrix_df = lupus_create_ct_df(
        log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_matrix_df = lupus_create_ct_df(
        inv_norm_log_pseudobulk_matrix,
        cell_type,
        donors
    )
    inv_norm_transformed_log_pseudobulk_model_residuals_df = lupus_create_ct_df(
        inv_norm_transformed_log_model_residuals,
        cell_type,
        donors
    )
    # assign processed cell-type data to dictionaries
    gene_pseudobulk_dics[cell_type] = log_pseudobulk_matrix_df
    gene_pseudobulk_invnorm_dics[cell_type] = inv_norm_transformed_log_pseudobulk_matrix_df
    gene_pseudobulk_residuals_invnorm_transformed_dics[cell_type] = inv_norm_transformed_log_pseudobulk_model_residuals_df


out_cell_type_log_pseudobulk_dir = "../data/lupus_cell_type_pseudobulk/log_tmm/log_pseudobulk_matrices"
makedirs(out_cell_type_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_dir = "../data/lupus_cell_type_pseudobulk/log_tmm/invnorm_log_pseudobulk_matrices"
makedirs(out_cell_type_inv_norm_log_pseudobulk_dir, exist_ok=True)

out_cell_type_inv_norm_log_pseudobulk_residuals_dir = "../data/lupus_cell_type_pseudobulk/log_tmm/invnorm_regression_residuals"
makedirs(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, exist_ok=True)


for ct in tqdm(reduced_cell_types):
    # process log tmm normalized
    gene_pseudobulk_dics[ct].to_csv(
        path.join(out_cell_type_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_invnorm_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )

    gene_pseudobulk_residuals_invnorm_transformed_dics[ct].to_csv(
        path.join(out_cell_type_inv_norm_log_pseudobulk_residuals_dir, f"{ct}.tsv"),
        sep="\t",
        index=True,
        index_label="gene_id"
    )


100%|██████████| 14/14 [14:54<00:00, 63.89s/it]
